In [2]:
path = "datasets/"
students = spark.read.csv(path+"students.csv", inferSchema=True, header=True)
tour = spark.read.csv(path+"pga_tour_historical.csv", inferSchema=True, header=True)
tour.limit(100).dropna().dropDuplicates().limit(5).show()
students.limit(5).toPandas()

+---------------+------+----------------+--------------------+-----+
|    Player Name|Season|       Statistic|            Variable|Value|
+---------------+------+----------------+--------------------+-----+
|Robert Garrigus|  2010|Driving Distance|Driving Distance ...|   71|
|   Bubba Watson|  2010|Driving Distance|Driving Distance ...|   77|
| Dustin Johnson|  2010|Driving Distance|Driving Distance ...|   83|
|Brett Wetterich|  2010|Driving Distance|Driving Distance ...|   54|
|    J.B. Holmes|  2010|Driving Distance|Driving Distance ...|  100|
+---------------+------+----------------+--------------------+-----+



,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


23/10/22 10:47:55 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


#### PandasUDFType
#### pandas_udf(f=None, returnType=None, functionType=None)

Creates a pandas user defined function (a.k.a. vectorized user defined function).

Pandas UDFs are user defined functions that are executed by Spark using Arrow to transfer data and Pandas to work with the data, which allows vectorized operations. A Pandas UDF is defined using the pandas_udf as a decorator or to wrap the function, and no additional configuration is required. A Pandas UDF behaves as a regular PySpark function API in general.

Parameters
* f – user-defined function. A python function if used as a standalone function
* returnType – the return type of the user-defined function. The value can be either a pyspark.sql.types.DataType object or a DDL-formatted type string.
* functionType – an enum value in pyspark.sql.functions.PandasUDFType. Default: SCALAR.

> Note This parameter exists for compatibility. Using Python type hints is encouraged.
In order to use this API, customarily the below are imported:

```
import pandas as pd
from pyspark.sql.functions import pandas_udf
```
From Spark 3.0 with Python 3.6+, Python type hints detect the function types as below:
```
@pandas_udf(IntegerType())
def slen(s: pd.Series) -> pd.Series:
    return s.str.len()
```
Prior to Spark 3.0, the pandas UDF used functionType to decide the execution type as below:

```
from pyspark.sql.functions import PandasUDFType
from pyspark.sql.types import IntegerType
@pandas_udf(IntegerType(), PandasUDFType.SCALAR)
def slen(s):
    return s.str.len()
```

In [3]:
# abs(col)
students.limit(5).select("*", abs(students["math score"]  * (0 - 1))).toPandas()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,abs((math score * -1))
0,female,group B,bachelor's degree,standard,none,72,72,74,72
1,female,group C,some college,standard,completed,69,90,88,69
2,female,group B,master's degree,standard,none,90,95,93,90
3,male,group A,associate's degree,free/reduced,none,47,57,44,47
4,male,group C,some college,standard,none,76,78,75,76


In [4]:
# acos(col)
# Returns inverse cosine of col, as if computed by java.lang.Math.acos()
students.limit(5).select("*", acos(0.01 * students["math score"])).toPandas()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,ACOS((math score * 0.01))
0,female,group B,bachelor's degree,standard,none,72,72,74,0.766994
1,female,group C,some college,standard,completed,69,90,88,0.809307
2,female,group B,master's degree,standard,none,90,95,93,0.451027
3,male,group A,associate's degree,free/reduced,none,47,57,44,1.081506
4,male,group C,some college,standard,none,76,78,75,0.707483


In [5]:
# add_month(start, months)
df = spark.createDataFrame([('2015-04-08',)], ['dt'])
df.select(add_months(df.dt, 1).alias('next_month')).collect()

[Row(next_month=datetime.date(2015, 5, 8))]

In [6]:
# approxCountDistinct(col, rsd=None)
# Deprecated in 2.1, use approx_count_distinct() instead.
# approx_count_distinct(col, rsd=None)
students.agg(approx_count_distinct(students["math score"]).alias('distinct_math_score')).show()

23/10/17 23:24:44 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------------+
|distinct_math_score|
+-------------------+
|                 82|
+-------------------+



In [7]:
# array(*cols)
# Creates a new array column.
students.select(array('math score', 'reading score').alias("arr")).show(5)

+--------+
|     arr|
+--------+
|[72, 72]|
|[69, 90]|
|[90, 95]|
|[47, 57]|
|[76, 78]|
+--------+
only showing top 5 rows



#### array_contains(col, value)
##### Collection function: returns null if the array is null, true if the array contains the given value, and false otherwise.
Parameters: 
- col – name of column containing array
- value – value or column to check for in array

In [8]:
df = spark.createDataFrame([(["a", "b", "c"],), ([],)], ['data'])
df.select(array_contains(df.data, "a")).collect()

[Row(array_contains(data, a)=True), Row(array_contains(data, a)=False)]

In [9]:
df.select(array_contains(df.data, lit("a"))).collect()

[Row(array_contains(data, a)=True), Row(array_contains(data, a)=False)]

In [10]:
# array_distinct(col)
# Collection function: removes duplicate values from the array.
df = spark.createDataFrame([([1, 2, 3, 2],), ([4, 5, 5, 4],)], ['data'])
df.select(array_distinct(df.data)).collect()

[Row(array_distinct(data)=[1, 2, 3]), Row(array_distinct(data)=[4, 5])]

In [11]:
# array_except(col1, col2)
# Collection function: returns an array of the elements in col1 but not in col2, without duplicates.
from pyspark.sql import Row
df = spark.createDataFrame([Row(c1=["b", "a", "c"], c2=["c", "d", "a", "f"])])
df.select(array_except(df.c1, df.c2)).collect()

[Row(array_except(c1, c2)=['b'])]

In [12]:
# array_intersect(col1, col2)
# Collection function: returns an array of the elements in the intersection of col1 and col2, without duplicates.
from pyspark.sql import Row
df = spark.createDataFrame([Row(c1=["b", "a", "c"], c2=["c", "d", "a", "f"])])
df.select(array_intersect(df.c1, df.c2)).collect()

[Row(array_intersect(c1, c2)=['a', 'c'])]

In [13]:
# array_join(col, delimiter, null_replacement=None)
# Concatenates the elements of column using the delimiter. Null values are replaced with null_replacement if set, otherwise they are ignored.
df = spark.createDataFrame([(["a", "b", "c"],), (["a", None],)], ['data'])
df.select(array_join(df.data, ",").alias("joined")).show()
df.select(array_join(df.data, ",", "NULL").alias("joined")).show()

+------+
|joined|
+------+
| a,b,c|
|     a|
+------+

+------+
|joined|
+------+
| a,b,c|
|a,NULL|
+------+



In [14]:
# array_max(col)
# Collection function: returns the maximum value of the array.
df = spark.createDataFrame([([2, 1, 3],), ([None, 10, -1],)], ['data'])
df.select(array_max(df.data).alias('max')).collect()

[Row(max=3), Row(max=10)]

In [15]:
# array_min(col)[source]
# Collection function: returns the minimum value of the array.
df = spark.createDataFrame([([2, 1, 3],), ([None, 10, -1],)], ['data'])
df.select(array_min(df.data).alias('min')).collect()

[Row(min=1), Row(min=-1)]

In [16]:
# array_position(col, value)
# Collection function: Locates the position of the first occurrence of the given value in the given array.
# Returns null if either of the arguments are null.
# The position is not zero based, but 1 based index.
# Returns 0 if the given value could not be found in the array.
df = spark.createDataFrame([(["c", "b", "a"],), ([],)], ['data'])
df.select(array_position(df.data, "a")).collect()

[Row(array_position(data, a)=3), Row(array_position(data, a)=0)]

In [17]:
# array_remove(col, element)
# Collection function: Remove all elements that equal to element from the given array.
df = spark.createDataFrame([([1, 2, 3, 1, 1],), ([],)], ['data'])
df.select(array_remove(df.data, 1)).collect()

[Row(array_remove(data, 1)=[2, 3]), Row(array_remove(data, 1)=[])]

In [18]:
# pyspark.sql.functions.array_repeat(col, count)
# Collection function: creates an array containing a column repeated count times.
df = spark.createDataFrame([('ab',)], ['data'])
df.select(array_repeat(df.data, 3).alias('r')).collect()

[Row(r=['ab', 'ab', 'ab'])]

In [19]:
# array_sort(col)
# Collection function: sorts the input array in ascending order. The elements of the input array must be orderable.
# Null elements will be placed at the end of the returned array.
df = spark.createDataFrame([([2, 1, None, 3],),([1],),([],)], ['data'])
df.select(array_sort(df.data).alias('r')).collect()

[Row(r=[1, 2, 3, None]), Row(r=[1]), Row(r=[])]

In [20]:
# array_union(col1, col2)
# Collection function: returns an array of the elements in the union of col1 and col2, without duplicates.
df = spark.createDataFrame([Row(c1=["b", "a", "c"], c2=["c", "d", "a", "f"])])
df.select(array_union(df.c1, df.c2)).collect()

[Row(array_union(c1, c2)=['b', 'a', 'c', 'd', 'f'])]

In [21]:
# arrays_overlap(a1, a2)
# Collection function: returns true if the arrays contain any common non-null element; if not, returns null if both the arrays are non-empty and any of them contains a null element; returns false otherwise.
df = spark.createDataFrame([(["a", "b"], ["b", "c"]), (["a"], ["b", "c"])], ['x', 'y'])
df.select(arrays_overlap(df.x, df.y).alias("overlap")).collect()

[Row(overlap=True), Row(overlap=False)]

In [22]:
# arrays_zip(*cols)
# Collection function: Returns a merged array of structs in which the N-th struct contains all N-th values of input arrays.
df = spark.createDataFrame([(([1, 2, 3], [2, 3, 4]))], ['vals1', 'vals2'])
df.select(arrays_zip(df.vals1, df.vals2).alias('zipped')).collect()

[Row(zipped=[Row(vals1=1, vals2=2), Row(vals1=2, vals2=3), Row(vals1=3, vals2=4)])]

In [23]:
# asc(col)¶
# Returns a sort expression based on the ascending order of the given column name.
students.select("math score", "reading score").orderBy(asc("reading score")).show(5)

+----------+-------------+
|math score|reading score|
+----------+-------------+
|         0|           17|
|        28|           23|
|        30|           24|
|         8|           24|
|        30|           26|
+----------+-------------+
only showing top 5 rows



In [24]:
# asc_nulls_first(col)¶
# Returns a sort expression based on the ascending order of the given column name, and null values return before non-null values.
students.select("math score", "reading score").orderBy(asc_nulls_first("reading score")).show(5)

+----------+-------------+
|math score|reading score|
+----------+-------------+
|         0|           17|
|        28|           23|
|        30|           24|
|         8|           24|
|        30|           26|
+----------+-------------+
only showing top 5 rows



In [25]:
# asc_nulls_last(col)¶
# Returns a sort expression based on the ascending order of the given column name, and null values appear after non-null values.
students.select("math score", "reading score").orderBy(asc_nulls_last("reading score")).show(5)

+----------+-------------+
|math score|reading score|
+----------+-------------+
|         0|           17|
|        28|           23|
|        30|           24|
|         8|           24|
|        30|           26|
+----------+-------------+
only showing top 5 rows



In [26]:
# ascii(col)
# Computes the numeric value of the first character of the string column.
tour.select("Player name", "Value", ascii("Player name")).show(5)

+---------------+-----+------------------+
|    Player name|Value|ascii(Player name)|
+---------------+-----+------------------+
|Robert Garrigus|   71|                82|
|   Bubba Watson|   77|                66|
| Dustin Johnson|   83|                68|
|Brett Wetterich|   54|                66|
|    J.B. Holmes|  100|                74|
+---------------+-----+------------------+
only showing top 5 rows



In [27]:
# asin(col)
# Returns inverse sine of col, as if computed by java.lang.Math.asin()
students.limit(5).select("*", asin(0.01 * students["math score"])).toPandas()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,ASIN((math score * 0.01))
0,female,group B,bachelor's degree,standard,none,72,72,74,0.803802
1,female,group C,some college,standard,completed,69,90,88,0.761489
2,female,group B,master's degree,standard,none,90,95,93,1.119770
3,male,group A,associate's degree,free/reduced,none,47,57,44,0.489291
4,male,group C,some college,standard,none,76,78,75,0.863313


23/10/17 23:24:51 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [28]:
# atan(col)
# Returns inverse tangent of col, as if computed by java.lang.Math.atan()
students.limit(5).select("*", atan(0.01 * students["math score"])).toPandas()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,ATAN((math score * 0.01))
0,female,group B,bachelor's degree,standard,none,72,72,74,0.624023
1,female,group C,some college,standard,completed,69,90,88,0.603983
2,female,group B,master's degree,standard,none,90,95,93,0.732815
3,male,group A,associate's degree,free/reduced,none,47,57,44,0.439361
4,male,group C,some college,standard,none,76,78,75,0.649870


In [29]:
# atan2(col1, col2)
# Returns the theta component of the point (r, theta) in polar coordinates that corresponds to the point (x, y) in Cartesian coordinates, as if computed by java.lang.Math.atan2()


In [30]:
# pyspark.sql.functions.avg(col)
# Aggregate function: returns the average of the values in a group.
students.filter(col("reading score") > 80).groupBy("gender").agg(avg(col("math score")).alias("average")).show()

+------+-----------------+
|gender|          average|
+------+-----------------+
|female|79.16981132075472|
|  male| 87.6842105263158|
+------+-----------------+



In [31]:
# base64(col)
# Computes the BASE64 encoding of a binary column and returns it as a string column.

In [32]:
# basestring
# alias of builtins.str

In [33]:
# bin(col)
# Returns the string representation of the binary value of the given column
tour.limit(5).select(bin(tour.Value).alias('c')).collect()

[Row(c='1000111'),
 Row(c='1001101'),
 Row(c='1010011'),
 Row(c='110110'),
 Row(c='1100100')]

In [34]:
# bitwiseNOT(col)¶
# Computes bitwise not.
data = [("Alice", 5), ("Bob", 10), ("Carol", 3)]
columns = ["Name", "Value"]
df = spark.createDataFrame(data, columns)
df.withColumn("BitwiseNOT_Value", bitwiseNOT(col("Value"))).show()

/Users/zsavchenko/.local/share/virtualenvs/spark_env-J_TJEM2Z/lib/python3.11/site-packages/pyspark/sql/functions.py:2083: FutureWarning: Deprecated in 3.2, use bitwise_not instead.
  warnings.warn("Deprecated in 3.2, use bitwise_not instead.", FutureWarning)


+-----+-----+----------------+
| Name|Value|BitwiseNOT_Value|
+-----+-----+----------------+
|Alice|    5|              -6|
|  Bob|   10|             -11|
|Carol|    3|              -4|
+-----+-----+----------------+



In [35]:
# broadcast(df)
# Marks a DataFrame as small enough for use in broadcast joins.


In [36]:
# bround(col, scale=0)
# Round the given value to scale decimal places using HALF_EVEN rounding mode if scale >= 0 or at integral part when scale < 0.
spark.createDataFrame([(2.5,)], ['a']).select(bround('a', 0).alias('r')).collect()

[Row(r=2.0)]

In [37]:
# cbrt(col)
# Computes the cube-root of the given value.
students.limit(5).select("*", cbrt(students["math score"])).toPandas()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,CBRT(math score)
0,female,group B,bachelor's degree,standard,none,72,72,74,4.160168
1,female,group C,some college,standard,completed,69,90,88,4.101566
2,female,group B,master's degree,standard,none,90,95,93,4.481405
3,male,group A,associate's degree,free/reduced,none,47,57,44,3.608826
4,male,group C,some college,standard,none,76,78,75,4.235824


In [38]:
# ceil(col)¶
# Computes the ceiling of the given value.
students.limit(5).select("*", ceil(students["math score"])).toPandas()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,CEIL(math score)
0,female,group B,bachelor's degree,standard,none,72,72,74,72
1,female,group C,some college,standard,completed,69,90,88,69
2,female,group B,master's degree,standard,none,90,95,93,90
3,male,group A,associate's degree,free/reduced,none,47,57,44,47
4,male,group C,some college,standard,none,76,78,75,76


In [39]:
# coalesce(*cols)
# Returns the first column that is not null.
cDf = spark.createDataFrame([(None, None), (1, None), (None, 2)], ("a", "b"))
cDf.show()
cDf.select(coalesce(cDf["a"], cDf["b"])).show()
cDf.select('*', coalesce(cDf["a"], lit(0.0))).show()

+----+----+
|   a|   b|
+----+----+
|NULL|NULL|
|   1|NULL|
|NULL|   2|
+----+----+

+--------------+
|coalesce(a, b)|
+--------------+
|          NULL|
|             1|
|             2|
+--------------+

+----+----+----------------+
|   a|   b|coalesce(a, 0.0)|
+----+----+----------------+
|NULL|NULL|             0.0|
|   1|NULL|             1.0|
|NULL|   2|             0.0|
+----+----+----------------+



In [40]:
# col(col)
# Returns a Column based on the given column name.
col("Value")

Column<'Value'>

In [41]:
# collect_list(col)¶
# Aggregate function: returns a list of objects with duplicates.
df2 = spark.createDataFrame([(2,), (5,), (5,)], ('age',))
df2.agg(collect_list('age')).collect()

[Row(collect_list(age)=[2, 5, 5])]

In [42]:
# collect_set(col)¶
# Aggregate function: returns a set of objects with duplicate elements eliminated.
df2 = spark.createDataFrame([(2,), (5,), (5,)], ('age',))
df2.agg(collect_set('age')).collect()

[Row(collect_set(age)=[5, 2])]

In [43]:
# column(col)¶
# Returns a Column based on the given column name.
column("Value")

Column<'Value'>

In [44]:
# concat(*cols)
# Concatenates multiple input columns together into a single column. The function works with strings, binary and compatible array columns.
df = spark.createDataFrame([('abcd','123')], ['s', 'd'])
df.select(concat(df.s, df.d).alias('s')).show()

df = spark.createDataFrame([([1, 2], [3, 4], [5]), ([1, 2], None, [3])], ['a', 'b', 'c'])
df.select(concat(df.a, df.b, df.c).alias("arr")).show()

+-------+
|      s|
+-------+
|abcd123|
+-------+

+---------------+
|            arr|
+---------------+
|[1, 2, 3, 4, 5]|
|           NULL|
+---------------+



In [45]:
# concat_ws(sep, *cols)
# Concatenates multiple input string columns together into a single string column, using the given separator.
df = spark.createDataFrame([('abcd','123')], ['s', 'd'])
df.select(concat_ws('-', df.s, df.d).alias('s')).collect()

[Row(s='abcd-123')]

In [46]:
# conv(col, fromBase, toBase)
# Convert a number in a string column from one base to another.
df = spark.createDataFrame([("010101",)], ['n'])
df.select(conv(df.n, 2, 16).alias('hex')).collect()


[Row(hex='15')]

In [47]:
# corr(col1, col2)
# Returns a new Column for the Pearson Correlation Coefficient for col1 and col2.
a = range(20)
b = [2 * x for x in range(20)]
df = spark.createDataFrame(zip(a, b), ["a", "b"])
df.agg(corr("a", "b").alias('c')).collect()

[Row(c=1.0)]

In [48]:
# cos(col)
# Parameters - col – angle in radians
# Returns - cosine of the angle, as if computed by java.lang.Math.cos().

In [49]:
# cosh(col)
# Parameters - col – hyperbolic angle
# Returns - hyperbolic cosine of the angle, as if computed by java.lang.Math.cosh()

In [50]:
# pyspark.sql.functions.count(col)
# Aggregate function: returns the number of items in a group.
students.select(count("math score")).show()

+-----------------+
|count(math score)|
+-----------------+
|             1000|
+-----------------+



In [51]:
# countDistinct(col, *cols)
# Returns a new Column for distinct count of col or cols.
tour.agg(countDistinct(tour.Value, tour["Player name"]).alias('c')).show()
tour.agg(countDistinct("Value", "Player name").alias('c')).show()

+-------+
|      c|
+-------+
|1054793|
+-------+



+-------+
|      c|
+-------+
|1054793|
+-------+



In [52]:
# covar_pop(col1, col2)
# Returns a new Column for the population covariance of col1 and col2.
a = [1] * 10
b = [1] * 10
df = spark.createDataFrame(zip(a, b), ["a", "b"])
df.agg(covar_pop("a", "b").alias('c')).collect()

[Row(c=0.0)]

In [53]:
# covar_samp(col1, col2)
# Returns a new Column for the sample covariance of col1 and col2.
a = [1] * 10
b = [1] * 10
df = spark.createDataFrame(zip(a, b), ["a", "b"])
df.agg(covar_samp("a", "b").alias('c')).collect()

[Row(c=0.0)]

In [54]:
# crc32(col)
# Calculates the cyclic redundancy check value (CRC32) of a binary column and returns the value as a bigint.
spark.createDataFrame([('ABC',)], ['a']).select(crc32('a').alias('crc32')).collect()

[Row(crc32=2743272264)]

In [55]:
# create_map(*cols)
# Creates a new map column.
# Parameters - cols – list of column names (string) or list of Column expressions that are grouped as key-value pairs, e.g. (key1, value1, key2, value2, …).

tour.select(create_map('Player name', 'Value').alias("map")).show(5, False)
tour.select(create_map([tour["Player name"], tour.Value]).alias("map")).show(5, False)

+-----------------------+
|map                    |
+-----------------------+
|{Robert Garrigus -> 71}|
|{Bubba Watson -> 77}   |
|{Dustin Johnson -> 83} |
|{Brett Wetterich -> 54}|
|{J.B. Holmes -> 100}   |
+-----------------------+
only showing top 5 rows

+-----------------------+
|map                    |
+-----------------------+
|{Robert Garrigus -> 71}|
|{Bubba Watson -> 77}   |
|{Dustin Johnson -> 83} |
|{Brett Wetterich -> 54}|
|{J.B. Holmes -> 100}   |
+-----------------------+
only showing top 5 rows



In [56]:
# cume_dist()
# Window function: returns the cumulative distribution of values within a window partition, i.e. the fraction of rows that are below the current row.


In [57]:
# current_date()
# Returns the current date as a DateType column.


In [58]:
# current_timestamp()
# Returns the current timestamp as a TimestampType column.

In [59]:
# date_add(start, days)
# Returns the date that is days days after start

df = spark.createDataFrame([('2015-04-08',)], ['dt'])
df.select(date_add(df.dt, 1).alias('next_date')).collect()

[Row(next_date=datetime.date(2015, 4, 9))]

In [60]:
# date_format(date, format)
# Converts a date/timestamp/string to a value of string in the format specified by the date format given by the second argument.
# A pattern could be for instance dd.MM.yyyy and could return a string like ‘18.03.1993’. All pattern letters of datetime pattern can be used.
# Use when ever possible specialized functions like year. These benefit from a specialized implementation.

df = spark.createDataFrame([('2015-04-08',)], ['dt'])
df.select(date_format('dt', 'MM/dd/yyy').alias('date')).collect()

[Row(date='04/08/2015')]

In [61]:
# date_sub(start, days)
# Returns the date that is days days before start
df = spark.createDataFrame([('2015-04-08',)], ['dt'])
df.select(date_sub(df.dt, 1).alias('prev_date')).show()

+----------+
| prev_date|
+----------+
|2015-04-07|
+----------+



In [62]:
# date_trunc(format, timestamp)
# Returns timestamp truncated to the unit specified by the format.
# Parameters - format – ‘year’, ‘yyyy’, ‘yy’, ‘month’, ‘mon’, ‘mm’, ‘day’, ‘dd’, ‘hour’, ‘minute’, ‘second’, ‘week’, ‘quarter’

df = spark.createDataFrame([('1997-02-28 05:02:11',)], ['t'])
df.select(date_trunc('year', df.t).alias('year')).show()

df.select(date_trunc('mon', df.t).alias('month')).show()

+-------------------+
|               year|
+-------------------+
|1997-01-01 00:00:00|
+-------------------+

+-------------------+
|              month|
+-------------------+
|1997-02-01 00:00:00|
+-------------------+



In [63]:
# datediff(end, start)
# Returns the number of days from start to end.

df = spark.createDataFrame([('2015-04-08','2015-05-10')], ['d1', 'd2'])
df.select(datediff(df.d2, df.d1).alias('diff')).collect()

[Row(diff=32)]

In [64]:
# dayofmonth(col)
# Extract the day of the month of a given date as integer.

df = spark.createDataFrame([('2015-04-08',)], ['dt'])
df.select(dayofmonth('dt').alias('day')).collect()

[Row(day=8)]

In [65]:
# dayofweek(col)
# Extract the day of the week of a given date as integer.

df = spark.createDataFrame([('2015-04-08',)], ['dt'])
df.select(dayofweek('dt').alias('day')).collect()

[Row(day=4)]

In [66]:
# dayofyear(col)
# Extract the day of the year of a given date as integer.

df = spark.createDataFrame([('2015-04-08',)], ['dt'])
df.select(dayofyear('dt').alias('day')).collect()

[Row(day=98)]

In [67]:
# decode(col, charset)
# Computes the first argument into a string from a binary using the provided character set (one of ‘US-ASCII’, ‘ISO-8859-1’, ‘UTF-8’, ‘UTF-16BE’, ‘UTF-16LE’, ‘UTF-16’).


In [68]:
# degrees(col)
# Converts an angle measured in radians to an approximately equivalent angle measured in degrees.
# Parameters - col – angle in radians
# Returns - angle in degrees, as if computed by java.lang.Math.toDegrees()


In [69]:
# dense_rank()
# Window function: returns the rank of rows within a window partition, without any gaps.
# The difference between rank and dense_rank is that dense_rank leaves no gaps in ranking sequence when there are ties. That is, if you were ranking a competition using dense_rank and had three people tie for second place, you would say that all three were in second place and that the next person came in third. Rank would give me sequential numbers, making the person that came in third place (after the ties) would register as coming in fifth.
# This is equivalent to the DENSE_RANK function in SQL.


In [70]:
# desc(col)
# Returns a sort expression based on the descending order of the given column name.
students.select("math score", "reading score").orderBy(desc("reading score")).show(5)

+----------+-------------+
|math score|reading score|
+----------+-------------+
|       100|          100|
|        92|          100|
|        97|          100|
|        87|          100|
|        87|          100|
+----------+-------------+
only showing top 5 rows



In [71]:
# desc_nulls_first(col)
# Returns a sort expression based on the descending order of the given column name, and null values appear before non-null values.
students.select("math score", "reading score").orderBy(desc_nulls_first("reading score")).show(5)

+----------+-------------+
|math score|reading score|
+----------+-------------+
|       100|          100|
|        92|          100|
|        97|          100|
|        87|          100|
|        87|          100|
+----------+-------------+
only showing top 5 rows



In [72]:
# desc_nulls_last(col)
# Returns a sort expression based on the descending order of the given column name, and null values appear after non-null values
students.select("math score", "reading score").orderBy(desc_nulls_last("reading score")).show(5)

+----------+-------------+
|math score|reading score|
+----------+-------------+
|       100|          100|
|        92|          100|
|        97|          100|
|        87|          100|
|        87|          100|
+----------+-------------+
only showing top 5 rows



In [73]:
# element_at(col, extraction)
# Collection function: Returns element of array at given index in extraction if col is array. Returns value for the given key in extraction if col is map.
# Parameters - col – name of column containing array or map
#            - extraction – index to check for in array or key to check for in map
# The position is not zero based, but 1 based index.

df = spark.createDataFrame([(["a", "b", "c"],), ([],)], ['data'])
df.select(element_at(df.data, 1)).show()

df = spark.createDataFrame([({"a": 1.0, "b": 2.0},), ({},)], ['data'])
df.select(element_at(df.data, lit("a"))).show()

+-------------------+
|element_at(data, 1)|
+-------------------+
|                  a|
|               NULL|
+-------------------+

+-------------------+
|element_at(data, a)|
+-------------------+
|                1.0|
|               NULL|
+-------------------+



In [74]:
# encode(col, charset)
# Computes the first argument into a binary from a string using the provided character set (one of ‘US-ASCII’, ‘ISO-8859-1’, ‘UTF-8’, ‘UTF-16BE’, ‘UTF-16LE’, ‘UTF-16’).


In [75]:
# exp(col)
# Computes the exponential of the given value.


In [76]:
# explode(col)
# Returns a new row for each element in the given array or map. Uses the default column name col for elements in the array and key and value for elements in the map unless specified otherwise.

eDF = spark.createDataFrame([Row(a=1, intlist=[1,2,3], mapfield={"a": "b"})])
eDF.select(explode(eDF.intlist).alias("anInt")).show()

eDF.select(explode(eDF.mapfield).alias("key", "value")).show()

+-----+
|anInt|
+-----+
|    1|
|    2|
|    3|
+-----+

+---+-----+
|key|value|
+---+-----+
|  a|    b|
+---+-----+



In [77]:
# explode_outer(col)
# Returns a new row for each element in the given array or map. Unlike explode, if the array/map is null or empty then null is produced. Uses the default column name col for elements in the array and key and value for elements in the map unless specified otherwise.

df = spark.createDataFrame(
    [(1, ["foo", "bar"], {"x": 1.0}), (2, [], {}), (3, None, None)],
    ("id", "an_array", "a_map")
)
df.select("id", "an_array", explode_outer("a_map")).show()
df.select("id", "a_map", explode_outer("an_array")).show()

+---+----------+----+-----+
| id|  an_array| key|value|
+---+----------+----+-----+
|  1|[foo, bar]|   x|  1.0|
|  2|        []|NULL| NULL|
|  3|      NULL|NULL| NULL|
+---+----------+----+-----+

+---+----------+----+
| id|     a_map| col|
+---+----------+----+
|  1|{x -> 1.0}| foo|
|  1|{x -> 1.0}| bar|
|  2|        {}|NULL|
|  3|      NULL|NULL|
+---+----------+----+



In [78]:
# expm1(col)
# Computes the exponential of the given value minus one.


In [79]:
# expr(str)
# Parses the expression string into the column that it represents

students.select(expr("length(gender)")).show(5)

+--------------+
|length(gender)|
+--------------+
|             6|
|             6|
|             6|
|             4|
|             4|
+--------------+
only showing top 5 rows



In [80]:
# factorial(col)
# Computes the factorial of the given value.

df = spark.createDataFrame([(5,)], ['n'])
df.select(factorial(df.n).alias('f')).collect()

[Row(f=120)]

In [81]:
# first(col, ignorenulls=False)[source]
# Aggregate function: returns the first value in a group.
# The function by default returns the first values it sees. It will return the first non-null value it sees when ignoreNulls is set to true. If all values are null, then null is returned.

first_value = tour.select(first("Value", ignorenulls=True)).alias("First_Value")
first_value

DataFrame[first(Value): string]

In [82]:
grouped_df = tour.groupBy("Player name").agg(first(col("Value")).alias("First_Value"))
grouped_df

DataFrame[Player name: string, First_Value: string]

In [83]:
first_value.show()
grouped_df.show(5)

+------------+
|first(Value)|
+------------+
|          71|
+------------+



+--------------+-----------+
|   Player name|First_Value|
+--------------+-----------+
|A.J. McInerney|          2|
|Aaron Baddeley|         94|
|Aaron Goldberg|          2|
| Aaron Krueger|          2|
|    Aaron Pike|         40|
+--------------+-----------+
only showing top 5 rows



In [84]:
# flatten(col)
# Collection function: creates a single array from an array of arrays. If a structure of nested arrays is deeper than two levels, only one level of nesting is removed.

df = spark.createDataFrame([([[1, 2, 3], [4, 5], [6]],), ([None, [4, 5]],)], ['data'])
df.select(flatten(df.data).alias('r')).collect()

[Row(r=[1, 2, 3, 4, 5, 6]), Row(r=None)]

In [85]:
# floor(col)
# Computes the floor of the given value.

df = spark.createDataFrame([(3.7,), (4.2,), (5.9,)], ["value"])
df.withColumn("floor_value", floor(df["value"])).show()

# df = spark.createDataFrame([("2023-10-09 14:30:45",), ("2023-10-09 14:45:30",)], ["timestamp"])
# df = df.withColumn("date", to_date(df["timestamp"]))
# df.withColumn("floor_date", floor(df["date"])).show()

df = spark.createDataFrame([(3.789,), (4.256,), (5.914,)], ["value"])
result_df = df.withColumn("floor_value", floor(df["value"] * 100) / 100).show()

+-----+-----------+
|value|floor_value|
+-----+-----------+
|  3.7|          3|
|  4.2|          4|
|  5.9|          5|
+-----+-----------+

+-----+-----------+
|value|floor_value|
+-----+-----------+
|3.789|       3.78|
|4.256|       4.25|
|5.914|       5.91|
+-----+-----------+



In [86]:
# format_number(col, d)
# Formats the number X to a format like ‘#,–#,–#.–’, rounded to d decimal places with HALF_EVEN round mode, and returns the result as a string.

spark.createDataFrame([(5,)], ['a']).select(format_number('a', 4).alias('v')).collect()

[Row(v='5.0000')]

In [87]:
# format_string(format, *cols)
# Formats the arguments in printf-style and returns the result as a string column.

df = spark.createDataFrame([(5, "hello")], ['a', 'b'])
df.select(format_string('%d %s', df.a, df.b).alias('v')).collect()

[Row(v='5 hello')]

In [88]:
# from_csv(col, schema, options={})
# Parses a column containing a CSV string to a row with the specified schema. Returns null, in the case of an unparseable string.

data = [("1,2,3",)]
df = spark.createDataFrame(data, ("value",))
df.select(from_csv(df.value, "a INT, b INT, c INT").alias("csv")).show()

value = data[0][0]
df.select(from_csv(df.value, schema_of_csv(value)).alias("csv")).show()

data = [("   abc",)]
df = spark.createDataFrame(data, ("value",))
options = {'ignoreLeadingWhiteSpace': True}
df.select(from_csv(df.value, "s string", options).alias("csv")).show()

+---------+
|      csv|
+---------+
|{1, 2, 3}|
+---------+

+---------+
|      csv|
+---------+
|{1, 2, 3}|
+---------+

+-----+
|  csv|
+-----+
|{abc}|
+-----+



In [90]:
# from_json(col, schema, options={})
# Parses a column containing a JSON string into a MapType with StringType as keys type, StructType or ArrayType with the specified schema. Returns null, in the case of an unparseable string.

data = [(1, '''{"a": 1}''')]
schema = StructType([StructField("a", IntegerType())])
df = spark.createDataFrame(data, ("key", "value"))
df.select(from_json(df.value, schema).alias("json")).show()

+----+
|json|
+----+
| {1}|
+----+



In [91]:
df.select(from_json(df.value, "a INT").alias("json")).show()

+----+
|json|
+----+
| {1}|
+----+



In [92]:
df.select(from_json(df.value, "MAP<STRING,INT>").alias("json")).show()

+--------+
|    json|
+--------+
|{a -> 1}|
+--------+



In [93]:
data = [(1, '''[{"a": 1}]''')]
schema = ArrayType(StructType([StructField("a", IntegerType())]))
df = spark.createDataFrame(data, ("key", "value"))
df.select(from_json(df.value, schema).alias("json")).show()

+-----+
| json|
+-----+
|[{1}]|
+-----+



In [94]:
schema = schema_of_json(lit('''{"a": 0}'''))
df.select(from_json(df.value, schema).alias("json")).show()

+------+
|  json|
+------+
|{NULL}|
+------+



In [95]:
data = [(1, '''[1, 2, 3]''')]
schema = ArrayType(IntegerType())
df = spark.createDataFrame(data, ("key", "value"))
df.select(from_json(df.value, schema).alias("json")).show()

+---------+
|     json|
+---------+
|[1, 2, 3]|
+---------+



In [96]:
# from_unixtime(timestamp, format='yyyy-MM-dd HH:mm:ss')
# Converts the number of seconds from unix epoch (1970-01-01 00:00:00 UTC) to a string representing the timestamp of that moment in the current system time zone in the given format.

spark.conf.set("spark.sql.session.timeZone", "America/Los_Angeles")
time_df = spark.createDataFrame([(1428476400,)], ['unix_time'])
time_df.select(from_unixtime('unix_time').alias('ts')).show()
spark.conf.unset("spark.sql.session.timeZone")

+-------------------+
|                 ts|
+-------------------+
|2015-04-08 00:00:00|
+-------------------+



#### rom_utc_timestamp(timestamp, tz)
This is a common function for databases supporting TIMESTAMP WITHOUT TIMEZONE. This function takes a timestamp which is timezone-agnostic, and interprets it as a timestamp in UTC, and renders that timestamp as a timestamp in the given time zone.

However, timestamp in Spark represents number of microseconds from the Unix epoch, which is not timezone-agnostic. So in Spark this function just shift the timestamp value from UTC timezone to the given timezone.

This function may return confusing result if the input is a string with timezone, e.g. ‘2018-03-13T06:18:23+00:00’. The reason is that, Spark firstly cast the string to timestamp according to the timezone in the string, and finally display the result by converting the timestamp to string according to the session local timezone.

Parameters:
* timestamp – the column that contains timestamps
* tz – A string detailing the time zone ID that the input should be adjusted to. It should be in the format of either region-based zone IDs or zone offsets. Region IDs must have the form ‘area/city’, such as ‘America/Los_Angeles’. Zone offsets must be in the format ‘(+|-)HH:mm’, for example ‘-08:00’ or ‘+01:00’. Also ‘UTC’ and ‘Z’ are supported as aliases of ‘+00:00’. Other short names are not recommended to use because they can be ambiguous.

tz can take a Column containing timezone ID strings.

In [97]:
df = spark.createDataFrame([('1997-02-28 10:30:00', 'JST')], ['ts', 'tz'])
df.select(from_utc_timestamp(df.ts, "PST").alias('local_time')).show()
df.select(from_utc_timestamp(df.ts, df.tz).alias('local_time')).show()

+-------------------+
|         local_time|
+-------------------+
|1997-02-28 02:30:00|
+-------------------+

+-------------------+
|         local_time|
+-------------------+
|1997-02-28 19:30:00|
+-------------------+



In [102]:
# get_json_object(col, path)
# Extracts json object from a json string based on json path specified, and returns json string of the extracted json object. It will return null if the input json string is invalid.

data = [("1", '''{"f1": "value1", "f2": "value2"}'''), ("2", '''{"f1": "value12"}''')]
df = spark.createDataFrame(data, ("key", "jstring"))
df.select(df.key, get_json_object(df.jstring, '$.f1').alias("c0"), get_json_object(df.jstring, '$.f2').alias("c1") ).show()

+---+-------+------+
|key|     c0|    c1|
+---+-------+------+
|  1| value1|value2|
|  2|value12|  NULL|
+---+-------+------+



In [101]:
# greatest(*cols)
# Returns the greatest value of the list of column names, skipping null values. This function takes at least 2 parameters. It will return null iff all parameters are null.

df = spark.createDataFrame([(1, 4, 3)], ['a', 'b', 'c'])
df.select(greatest(df.a, df.b, df.c).alias("greatest")).show()

+--------+
|greatest|
+--------+
|       4|
+--------+



In [104]:
# grouping(col)
# Aggregate function: indicates whether a specified column in a GROUP BY list is aggregated or not, returns 1 for aggregated or 0 for not aggregated in the result set.

tour.cube("Player name").agg(grouping("Player name"), sum("Value")).orderBy("Player name").show()

+--------------+---------------------+--------------------+
|   Player name|grouping(Player name)|          sum(Value)|
+--------------+---------------------+--------------------+
|          NULL|                    1|3.221497314902330...|
|A.J. McInerney|                    0|          4202016.71|
|Aaron Baddeley|                    0|1.5230582441420002E9|
|Aaron Goldberg|                    0|         8062168.635|
| Aaron Krueger|                    0|           4029172.0|
|    Aaron Pike|                    0|  185.00000000000003|
|     Aaron Rai|                    0|              541.57|
|Aaron Townsend|                    0|          4025596.52|
| Aaron Watkins|                    0|      1.2073087502E7|
|    Aaron Wise|                    0| 8.629364958069998E8|
| Abraham Ancer|                    0| 9.412399311720005E8|
|    Adam Bland|                    0|1.0204229770000001E7|
|    Adam Blyth|                    0|              412.09|
|Adam Cornelson|                    0|  

In [105]:
# grouping_id(*cols)
# Aggregate function: returns the level of grouping, equals to (grouping(c1) << (n-1)) + (grouping(c2) << (n-2)) + … + grouping(cn)
# Note – The list of columns should match with grouping columns exactly, or empty (means all the grouping columns).

tour.cube("Player name").agg(grouping_id(), sum("Value")).orderBy("Player name").show()

+--------------+-------------+--------------------+
|   Player name|grouping_id()|          sum(Value)|
+--------------+-------------+--------------------+
|          NULL|            1|3.221497314902330...|
|A.J. McInerney|            0|          4202016.71|
|Aaron Baddeley|            0|1.5230582441420002E9|
|Aaron Goldberg|            0|         8062168.635|
| Aaron Krueger|            0|           4029172.0|
|    Aaron Pike|            0|  185.00000000000003|
|     Aaron Rai|            0|              541.57|
|Aaron Townsend|            0|          4025596.52|
| Aaron Watkins|            0|      1.2073087502E7|
|    Aaron Wise|            0| 8.629364958069998E8|
| Abraham Ancer|            0| 9.412399311720005E8|
|    Adam Bland|            0|1.0204229770000001E7|
|    Adam Blyth|            0|              412.09|
|Adam Cornelson|            0|          4033392.96|
| Adam Crawford|            0|   570.2600000000001|
| Adam D'Amario|            0|           4037064.0|
|      Adam 

In [107]:
# hash(*cols)
# Calculates the hash code of given columns, and returns the result as an int column.

spark.createDataFrame([('ABC',)], ['a']).select(hash('a').alias('hash')).collect()

[Row(hash=-757602832)]

In [108]:
# hex(col)¶
# Computes hex value of the given column, which could be pyspark.sql.types.StringType, pyspark.sql.types.BinaryType, pyspark.sql.types.IntegerType or pyspark.sql.types.LongType.

spark.createDataFrame([('ABC', 3)], ['a', 'b']).select(hex('a'), hex('b')).collect()

[Row(hex(a)='414243', hex(b)='3')]

In [109]:
# hour(col)
# Extract the hours of a given date as integer.

df = spark.createDataFrame([('2015-04-08 13:08:15',)], ['ts'])
df.select(hour('ts').alias('hour')).collect()

[Row(hour=13)]

In [ ]:
# hypot(col1, col2)
# Computes sqrt(a^2 + b^2) without intermediate overflow or underflow.



In [110]:
# initcap(col)
# Translate the first letter of each word to upper case in the sentence.

spark.createDataFrame([('ab cd',)], ['a']).select(initcap("a").alias('v')).collect()

[Row(v='Ab Cd')]

In [ ]:
# input_file_name()
# Creates a string column for the file name of the current Spark task.



In [111]:
# instr(str, substr)
# Locate the position of the first occurrence of substr column in the given string. Returns null if either of the arguments are null.
# Note – The position is not zero based, but 1 based index. Returns 0 if substr could not be found in str.

df = spark.createDataFrame([('abcd',)], ['s',])
df.select(instr(df.s, 'b').alias('s')).collect()

[Row(s=2)]

In [112]:
# isnan(col)
# An expression that returns true iff the column is NaN.

df = spark.createDataFrame([(1.0, float('nan')), (float('nan'), 2.0)], ("a", "b"))
df.select(isnan("a").alias("r1"), isnan(df.a).alias("r2")).collect()

[Row(r1=False, r2=False), Row(r1=True, r2=True)]

In [113]:
# isnull(col)
# An expression that returns true iff the column is null.

df = spark.createDataFrame([(1, None), (None, 2)], ("a", "b"))
df.select(isnull("a").alias("r1"), isnull(df.a).alias("r2")).collect()

[Row(r1=False, r2=False), Row(r1=True, r2=True)]

In [114]:
# json_tuple(col, *fields)
# Creates a new row for a json column according to the given field names.

data = [("1", '''{"f1": "value1", "f2": "value2"}'''), ("2", '''{"f1": "value12"}''')]
df = spark.createDataFrame(data, ("key", "jstring"))
df.select(df.key, json_tuple(df.jstring, 'f1', 'f2')).collect()

[Row(key='1', c0='value1', c1='value2'), Row(key='2', c0='value12', c1=None)]

In [ ]:
# kurtosis(col)
# Aggregate function: returns the kurtosis of the values in a group.



In [ ]:
# lag(col, offset=1, default=None)
# This is equivalent to the LAG function in SQL.
# Window function: returns the value that is offset rows before the current row, and defaultValue if there is less than offset rows before the current row. For example, an offset of one will return the previous row at any given point in the window partition.



In [ ]:
# last(col, ignorenulls=False)
# Aggregate function: returns the last value in a group.
# The function by default returns the last values it sees. It will return the last non-null value it sees when ignoreNulls is set to true. If all values are null, then null is returned.



In [115]:
# last_day(date)[source]
# Returns the last day of the month which the given date belongs to.

df = spark.createDataFrame([('1997-02-10',)], ['d'])
df.select(last_day(df.d).alias('date')).collect()

[Row(date=datetime.date(1997, 2, 28))]

In [ ]:
# lead(col, offset=1, default=None)
# This is equivalent to the LEAD function in SQL.
# Window function: returns the value that is offset rows after the current row, and defaultValue if there is less than offset rows after the current row. For example, an offset of one will return the next row at any given point in the window partition.



In [116]:
# least(*cols)
# Returns the least value of the list of column names, skipping null values. This function takes at least 2 parameters. It will return null iff all parameters are null.

df = spark.createDataFrame([(1, 4, 3)], ['a', 'b', 'c'])
df.select(least(df.a, df.b, df.c).alias("least")).collect()

[Row(least=1)]

In [117]:
# length(col)
# Computes the character length of string data or number of bytes of binary data. The length of character data includes the trailing spaces. The length of binary data includes binary zeros.

spark.createDataFrame([('ABC ',)], ['a']).select(length('a').alias('length')).collect()

[Row(length=4)]

In [ ]:
# levenshtein(left, right)
# Computes the Levenshtein distance of the two given strings.

df0 = spark.createDataFrame([('kitten', 'sitting',)], ['l', 'r'])
df0.select(levenshtein('l', 'r').alias('d')).collect()

In [ ]:
# lit(col)
# Creates a Column of literal value.

df.select(lit(5).alias('height')).withColumn('spark_user', lit(True)).take(1)

#### locate(substr, str, pos=1)
Locate the position of the first occurrence of substr in a string column, after position pos.

Note The position is not zero based, but 1 based index. Returns 0 if substr could not be found in str.

Parameters:
* substr – a string
* str – a Column of pyspark.sql.types.StringType
* pos – start position (zero based)

In [ ]:
df = spark.createDataFrame([('abcd',)], ['s',])
df.select(locate('b', df.s, 1).alias('s')).collect()

In [123]:
# og(arg1, arg2=None)
# Returns the first argument-based logarithm of the second argument.
# If there is only one argument, then this takes the natural logarithm of the argument.

students.limit(5).select(log(10.0, students["math score"]).alias('ten')).rdd.map(lambda l: str(l.ten)[:7]).collect()

['1.85733', '1.83884', '1.95424', '1.67209', '1.88081']

In [122]:
students.limit(5).select(log(students["math score"]).alias('e')).rdd.map(lambda l: str(l.e)[:7]).collect()

['4.27666', '4.23410', '4.49980', '3.85014', '4.33073']

In [118]:
# log10(col)
# Computes the logarithm of the given value in Base 10.


# log1p(col)
# Computes the natural logarithm of the given value plus one.


# log2(col)
# Returns the base-2 logarithm of the argument.

spark.createDataFrame([(4,)], ['a']).select(log2('a').alias('log2')).collect()

[Row(log2=2.0)]

In [ ]:
# lower(col)
# Converts a string expression to lower case.



In [119]:
# lpad(col, len, pad)
# Left-pad the string column to width len with pad.

df = spark.createDataFrame([('abcd',)], ['s',])
df.select(lpad(df.s, 6, '#').alias('s')).collect()

[Row(s='##abcd')]